In [3]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GRU, Flatten,MaxPooling1D,BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model,model_from_json


In [4]:
d1 = pd.read_csv('../datas/dos.csv')
d2 = pd.read_csv('../datas/normal.csv')

In [5]:
d1.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,10.0.2.15,192.168.56.1,TCP,54,2837 > 8000 [SYN] Seq=0 Win=512 Len=0
1,2,0.000875,192.168.56.1,10.0.2.15,TCP,60,"8000 > 2837 [SYN, ACK] Seq=0 Ack=1 Win=65535..."
2,3,0.000915,10.0.2.15,192.168.56.1,TCP,54,2837 > 8000 [RST] Seq=1 Win=0 Len=0
3,4,1.000401,10.0.2.15,192.168.56.1,TCP,54,2838 > 8000 [SYN] Seq=0 Win=512 Len=0
4,5,1.001788,192.168.56.1,10.0.2.15,TCP,60,"8000 > 2838 [SYN, ACK] Seq=0 Ack=1 Win=65535..."


In [7]:
d2.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,10.0.2.15,52.223.2.229,TLSv1.2,93,Application Data
1,2,0.000760,52.223.2.229,10.0.2.15,TCP,60,443 > 58034 [ACK] Seq=1 Ack=40 Win=65535 Len=0
2,3,0.059263,52.223.2.229,10.0.2.15,TLSv1.2,93,Application Data
3,4,0.059310,10.0.2.15,52.223.2.229,TCP,54,58034 > 443 [ACK] Seq=40 Ack=40 Win=31064 Len=0
4,5,0.378067,10.0.2.15,23.48.226.59,TCP,54,53346 > 443 [ACK] Seq=1 Ack=1 Win=31240 Len=0


In [6]:
d2[d2['Info'] == 'Application Data']

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,10.0.2.15,52.223.2.229,TLSv1.2,93,Application Data
2,3,0.059263,52.223.2.229,10.0.2.15,TLSv1.2,93,Application Data
8,9,3.001482,10.0.2.15,104.18.38.76,TLSv1.2,93,Application Data
10,11,3.029429,104.18.38.76,10.0.2.15,TLSv1.2,93,Application Data
14,15,4.002840,10.0.2.15,13.33.146.21,TLSv1.2,93,Application Data
...,...,...,...,...,...,...,...
769,770,15.575767,10.0.2.15,34.95.69.49,TLSv1.3,141,Application Data
770,771,15.575958,10.0.2.15,34.95.69.49,TLSv1.3,575,Application Data
781,782,15.862257,10.0.2.15,27.123.42.204,TLSv1.3,157,Application Data
783,784,15.879032,34.95.69.49,10.0.2.15,TLSv1.3,197,Application Data


In [7]:
d2['Info'].unique()

array(['Application Data',
       '443  >  58034 [ACK] Seq=1 Ack=40 Win=65535 Len=0',
       '58034  >  443 [ACK] Seq=40 Ack=40 Win=31064 Len=0',
       '53346  >  443 [ACK] Seq=1 Ack=1 Win=31240 Len=0',
       '[TCP ACKed unseen segment] 443  >  53346 [ACK] Seq=1 Ack=2 Win=65535 Len=0',
       '33042  >  80 [ACK] Seq=1 Ack=1 Win=31240 Len=0',
       '[TCP ACKed unseen segment] 80  >  33042 [ACK] Seq=1 Ack=2 Win=65535 Len=0',
       '443  >  37554 [ACK] Seq=1 Ack=40 Win=65535 Len=0',
       '37554  >  443 [ACK] Seq=40 Ack=40 Win=31240 Len=0',
       '55404  >  80 [ACK] Seq=1 Ack=1 Win=31691 Len=0',
       '[TCP ACKed unseen segment] 80  >  55404 [ACK] Seq=1 Ack=2 Win=65535 Len=0',
       '443  >  37614 [ACK] Seq=1 Ack=40 Win=65535 Len=0',
       '37614  >  443 [ACK] Seq=40 Ack=40 Win=31064 Len=0',
       '443  >  44994 [ACK] Seq=1 Ack=40 Win=65535 Len=0',
       '[TCP Previous segment not captured] 33042  >  80 [FIN, ACK] Seq=2 Ack=1 Win=31240 Len=0',
       '80  >  33042 [ACK] Seq=1 A

In [8]:
d1.shape

(801, 7)

In [9]:
d1['type'] = 'ddos'

# Save the updated DataFrame back to CSV
# d1.to_csv('dataset_with_type_column.csv', index=False)

In [10]:
d1.shape

(801, 8)

In [11]:
d2['type'] = 'normal'

In [18]:
combined_df = pd.concat([d1, d2], ignore_index=True)


In [15]:
# combined_df.to_csv('combined_dataset.csv', index=False)


In [39]:
cf = pd.read_csv('combined_dataset.csv')
cf.head()

,No.,Time,Source,Destination,Protocol,Length,Info,type
0,1,0.000000,10.0.2.15,192.168.56.1,TCP,54,2837 > 8000 [SYN] Seq=0 Win=512 Len=0,ddos
1,2,0.000875,192.168.56.1,10.0.2.15,TCP,60,"8000 > 2837 [SYN, ACK] Seq=0 Ack=1 Win=65535...",ddos
2,3,0.000915,10.0.2.15,192.168.56.1,TCP,54,2837 > 8000 [RST] Seq=1 Win=0 Len=0,ddos
3,4,1.000401,10.0.2.15,192.168.56.1,TCP,54,2838 > 8000 [SYN] Seq=0 Win=512 Len=0,ddos
4,5,1.001788,192.168.56.1,10.0.2.15,TCP,60,"8000 > 2838 [SYN, ACK] Seq=0 Ack=1 Win=65535...",ddos


In [14]:
cf.shape

(1591, 8)

In [40]:
cf['Protocol'].unique()

array(['TCP', 'ARP', 'TLSv1.2', 'DNS', 'TLSv1.3', 'OCSP', 'QUIC'],
      dtype=object)

In [70]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder



# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit-label-encode 'Protocol' column
cf['Protocol'] = label_encoder.fit_transform(cf['Protocol'])
cf['type'] = label_encoder.fit_transform(cf['type'])


# Print the encoded dataframe
# print(df)


In [71]:
cf.head()

,No.,Time,Source,Destination,Protocol,Length,Info,type
0,1,0.000000,10.0.2.15,192.168.56.1,4,54,2837 > 8000 [SYN] Seq=0 Win=512 Len=0,0
1,2,0.000875,192.168.56.1,10.0.2.15,4,60,"8000 > 2837 [SYN, ACK] Seq=0 Ack=1 Win=65535...",0
2,3,0.000915,10.0.2.15,192.168.56.1,4,54,2837 > 8000 [RST] Seq=1 Win=0 Len=0,0
3,4,1.000401,10.0.2.15,192.168.56.1,4,54,2838 > 8000 [SYN] Seq=0 Win=512 Len=0,0
4,5,1.001788,192.168.56.1,10.0.2.15,4,60,"8000 > 2838 [SYN, ACK] Seq=0 Ack=1 Win=65535...",0


In [45]:
cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No.          1591 non-null   int64  
 1   Time         1591 non-null   float64
 2   Source       1591 non-null   object 
 3   Destination  1591 non-null   object 
 4   Protocol     1591 non-null   int32  
 5   Length       1591 non-null   int64  
 6   Info         1591 non-null   object 
 7   type         1591 non-null   object 
dtypes: float64(1), int32(1), int64(2), object(4)
memory usage: 93.3+ KB


In [48]:
cf.isna().sum()

No.            0
Time           0
Source         0
Destination    0
Protocol       0
Length         0
Info           0
type           0
dtype: int64

In [49]:
cf.shape

(1591, 8)

In [72]:
cf['type']

0       0
1       0
2       0
3       0
4       0
       ..
1586    1
1587    1
1588    1
1589    1
1590    1
Name: type, Length: 1591, dtype: int32

In [73]:
X = cf.drop(columns=['No.','Source','Destination','type','Info'])
y = cf['type']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [75]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1113, 3)
X_test shape: (478, 3)
y_train shape: (1113,)
y_test shape: (478,)


In [76]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train_sc = scale.fit_transform(X_train)
X_test_sc = scale.fit_transform(X_test)

In [77]:
X_train_reshape = np.reshape(X_train_sc, (X_train_sc.shape[0], X_train_sc.shape[1], 1))
X_train_reshape.shape


X_test_reshape = X_test_sc.reshape(X_test_sc.shape[0],X_test_sc.shape[1],1)
X_test_reshape.shape

(478, 3, 1)

In [78]:
model = Sequential([
    Conv1D(64,3,activation='relu',padding='same',kernel_regularizer=l2(0.03),input_shape=(X_train_reshape.shape[1],1)),
    BatchNormalization(),
    Dropout(0.2),
    GRU(128,activation='relu',return_sequences=True),
    Flatten(),
    Dense(1,activation='sigmoid')
])

In [79]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=(['accuracy']))


In [80]:
# Training the model

history = model.fit(X_train_reshape,y_train,batch_size=120,epochs=5,validation_data=(X_test_reshape,y_test))

Epoch 1/5
10/10 [==============================] - 6s 115ms/step - loss: 0.5707 - accuracy: 0.7745 - val_loss: 0.6971 - val_accuracy: 0.7657
Epoch 2/5
10/10 [==============================] - 0s 21ms/step - loss: 0.3396 - accuracy: 0.9003 - val_loss: 0.6461 - val_accuracy: 0.8347
Epoch 3/5
10/10 [==============================] - 0s 23ms/step - loss: 0.2611 - accuracy: 0.9218 - val_loss: 0.5989 - val_accuracy: 0.9121
Epoch 4/5
10/10 [==============================] - 0s 25ms/step - loss: 0.2265 - accuracy: 0.9398 - val_loss: 0.5601 - val_accuracy: 0.9351
Epoch 5/5
10/10 [==============================] - 0s 19ms/step - loss: 0.2144 - accuracy: 0.9506 - val_loss: 0.5336 - val_accuracy: 0.9456


In [81]:
# model.save('det_model.h5')


In [84]:
y_pred = model.predict(X_test_sc)

15/15 [==============================] - 12s 28ms/step


In [85]:
y_pred = (y_pred > 0.5).astype(int) 

In [87]:
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

f1 = f1_score(y_test,y_pred)
print('F1-Score: ',f1)

Accuracy: 0.9456066945606695
F1-Score:  0.9467213114754099


In [88]:
#Confsion matrix
cm = confusion_matrix(y_test,y_pred)
print('Confusion Matrix:\n',cm)

Confusion Matrix:
 [[221  26]
 [  0 231]]


In [89]:
# Plot of confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()